In [1]:
import os, sys
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import time
import json
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import awswrangler as wr

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
utils_path = os.path.join('/home/ec2-user/SageMaker/thinktank_casestudy/src/utils/')
pp_path = os.path.join('/home/ec2-user/SageMaker/thinktank_casestudy/src/preprocessing/')

sys.path.append(utils_path)
sys.path.append(pp_path)

In [3]:
import preprocessing
import utils

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [4]:
prefix_path = 's3://think-tank-casestudy/preprocessed_data'

In [5]:
n_cluster = 4000

In [6]:
train_data = wr.s3.read_parquet(path=f'{prefix_path}/n_cluster_{n_cluster}/train_data_clustered.parquet')
test_data = wr.s3.read_parquet(path=f'{prefix_path}/n_cluster_{n_cluster}/test_data_clustered.parquet')

- CALL TYPE -> ONE_HOT ENCODING, no ordinal relationship
- WEATHER --> ONE HOT ENCODING, no ordinal relationship
- ORIGIN STAND --> Reduction of High cardinality + ONE HOT ENCODING
- MONTH/WEEK per year --> ONE HOT ENCODING or ORDINAL ENCODING

In [7]:
train_data = utils.extend_timestamps(train_data)
test_data = utils.extend_timestamps(test_data)

In [8]:
train_data = utils.reduce_high_cardinality(train_data, ['ORIGIN_STAND'])
test_data = utils.reduce_high_cardinality(test_data, ['ORIGIN_STAND'])

In [9]:
categories_oh = ['CALL_TYPE','WEATHER','ORIGIN_STAND_agg','YEAR_MONTH']

In [10]:
df_fenc_oh = utils.feature_encoding_oh(train_data, categories_oh)
train_data = pd.concat([train_data, df_fenc_oh],axis=1)

In [11]:
df_fenc_oh = utils.feature_encoding_oh(test_data, categories_oh)
test_data = pd.concat([test_data, df_fenc_oh],axis=1)

In [12]:
test_data, train_data = utils.add_binary_features(train_data, test_data)

In [13]:
assert(train_data.shape[1] == test_data.shape[1])

In [14]:
non_features = ['CALL_TYPE','ORIGIN_CALL','ORIGIN_STAND', 'START_POINT','DEST_POINT',
                'TIMESTAMP_MONTH','TIMESTAMP_DAY','TIMESTAMP_WEEK','TIMESTAMP_YEAR','YEAR_MONTH','ORIGIN_STAND_agg',
               'MISSING_DATA','POLYLINE','WEATHER','TAXI_ID','TIMESTAMP_DT','TIMESTAMP']

In [15]:
train_data = train_data[[column_ for column_ in train_data.columns if column_ not in non_features]]
test_data = test_data[[column_ for column_ in train_data.columns if column_ not in non_features]]

In [16]:
n_cluster =train_data.CLUSTER_LABEL.nunique()

In [17]:
wr.s3.to_parquet(df=train_data, index=train_data.index, path=f's3://think-tank-casestudy/features_engineered/n_cluster_{n_cluster}/feature_engineered_train.parquet')
wr.s3.to_parquet(df=test_data, index=test_data.index, path=f's3://think-tank-casestudy/features_engineered/n_cluster_{n_cluster}/feature_engineered_test.parquet')

{'paths': ['s3://think-tank-casestudy/features_engineered/n_cluster_4000/feature_engineered_test.parquet'],
 'partitions_values': {}}